In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import unidecode
import time

1. 访问次数--done
2. 举报人信息：
    * 共几人举报--done
3. 被举报人信息：
    * 用户名--done
    * 性别--not wanted
    * 位置--done
    * 信用等级--done
4. 举报人陈述时间：
    * 举报人--not wanted
    * 举报信息--not wanted
    * 举报时间--not wanted
5. 被举报人微博：
    * 被举报人--done
    * 原微博
        * 内容
        * 发布时间--done
        * 点赞/转发数
        * 评论数/评论内容

In [2]:
path = []
for i in range(1, 51):
    path.append('text_data/' + str(i) + '.txt')

In [3]:
for i in range(0, 50):
    crawl_path = 'crawl_data/text_data' + str(i) + '.csv'
    html = open(path[i], 'r', encoding='utf-8')
    htmlhandle = html.read()
    soup = BeautifulSoup(htmlhandle, 'lxml')
    
    result = pd.DataFrame()
    result.drop(result.index,inplace=True)
    view = []
    inform = []
    reported_id = []
    reported_address = []
    reported_gender = []
    reported_credit = []
    post_time = []
    cn_links = []
    ## 访问次数
    li = soup.find_all(text=re.compile("访问次数"))
    pattern = re.compile(r'\d+')
    for item in li:
        string = unidecode.unidecode(item.string)
        view.append(re.findall(pattern, string)[0])
    ## 举报人数
    list_report = soup.select('input[node-type="uids_num"] ')
    for item in list_report:
        inform.append(item['value'])

    for item in soup.find_all("div", class_="user bg_orange2 clearfix"):
        ## 被举报人id
        link_u = item.select('p a')
        reported_id.append(str(link_u)[29:39])
    
        ## 被举报人位置
        string = str(item.contents[5])[-15:]
        add = re.sub("[A-Za-z0-9\<\>\"\/\\t]", "", string)
        reported_address.append(add)
    
        ## 被举报人信用等级
        x = item.select('p a[target="_blank"] ')
        credit_u_i = re.findall(r'信用等级：(.+)"' ,str(x))
        reported_credit.append(credit_u_i)
    
    ## 原微博发布时间
    '''for item in soup.find_all(text=re.compile("发布时间")):
        match=re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', item)
        if match==None:
            post_time.append('nn')
        else:
            post_time.append(time.strptime(match.group(), '%Y-%m-%d %H:%M:%S'))'''
    for item in soup.find_all('div', {'class': 'item top'}):
        if item.p.a == None:
            post_time.append('None')
        else:
            match = re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',str(item.p))
            post_time.append(match[0])
            #post_time.append(time.strptime(match.group(), '%Y-%m-%d %H:%M:%S'))
    
    ## 原微博链接
    for item in soup.select('div[class="W_main_half_r"] div div div div[class="item top"]'):
        if item.p.a==None:
            cn_links.append('nn')
        else:
            weibo_cn = re.findall(r'(?<=\/)[\w]*(?=\"\s)', str(item.p.a))
            cn_links.append('https://weibo.cn/comment/' + weibo_cn[0])
    result['view_num'] = view
    result['inform_num'] = inform
    result['reported_id'] = reported_id
    '''result['reported_gender'] = reported_gender'''
    result['reported_address'] = reported_address
    result['post_time'] = post_time
    result['link_to_post'] = cn_links
    result['reported_credit'] = reported_credit
    print(result)
    result.to_csv (crawl_path, index = None, header=True)

   view_num inform_num reported_id reported_address            post_time  \
0        90          1  3201509481               北京  2019-08-03 15:35:09   
1        83          5  6183832861               安徽  2019-08-22 12:25:44   
2       259          2  1626336345            河北 保定  2019-08-20 20:03:08   
3        99          1  5074648251               北京                 None   
4        73          2  6107054710            山东 东营  2019-08-22 11:28:58   
5       389         45  5185572214               河南  2019-08-22 15:40:24   
6       104          2  1909702753               上海  2019-08-27 19:03:09   
7       183          6  5616413326               广东  2019-08-23 07:29:49   
8        67          1  2742655797               其他  2019-08-29 10:28:28   
9        88          2  1916653805               浙江  2019-08-29 02:04:46   
10       67          1  2260417513               海外  2019-08-23 13:35:39   
11       68          1  1163390501               上海  2019-08-27 22:47:16   
12       64 

   view_num inform_num reported_id reported_address            post_time  \
0        74          1  5978812295               北京  2019-07-26 14:43:06   
1        23          3  6805175458            海外 英国  2019-07-25 20:26:32   
2        27          1  2184232104               香港  2019-07-26 21:47:50   
3        19          1  2767558427            四川 成都  2019-07-24 13:16:15   
4        35          1  5293292799               浙江  2019-07-28 12:49:21   
5        29          2  1202586970               北京  2019-07-26 07:33:07   
6        15          1  3045706314               陕西  2019-07-30 13:36:17   
7       155          3  5662234502               海外                 None   
8        24          3  6622625535               天津  2019-07-30 13:46:44   
9       100         32  5692541220               四川  2019-07-26 02:15:42   
10       41          1  6020737496               陕西                 None   
11       27          2  1760778834            江苏 南京                 None   
12       33 

   view_num inform_num reported_id reported_address            post_time  \
0        98          1  3741288143               四川  2019-07-11 11:22:45   
1        67          1  2542384550            安徽 合肥  2019-07-11 20:23:14   
2        33          1  1843324043               北京  2019-07-02 21:34:45   
3        30          1  5743527870               陕西  2019-07-05 17:27:41   
4        96         10  5764432942               辽宁  2019-07-08 21:16:24   
5        39          2  3018192657               湖北  2019-07-08 20:33:11   
6        31          1  6069152763               吉林  2019-05-11 09:04:27   
7        40          1  5634644882               福建                 None   
8       745          1  2232193862               湖南  2019-07-09 23:08:06   
9        64          6  3486040104               福建  2019-07-09 12:30:13   
10      121          2  6553771110               其他  2019-07-10 13:42:34   
11      435         40  3226899882               香港  2019-07-09 21:41:02   
12      133 

   view_num inform_num reported_id reported_address            post_time  \
0        27          1  1367679907               其他  2019-06-26 12:19:13   
1        59          1  1221684892               江苏  2019-06-27 12:17:39   
2        27          3  6626161298               辽宁  2019-06-28 10:03:12   
3        22          1  6560162768               其他  2019-06-28 11:00:50   
4        20          1  6525535933               广东  2019-06-26 18:38:34   
5        31          2  3177920131            广东 广州  2019-06-27 22:41:45   
6        69          3  1297870702            河北 沧州  2019-06-27 22:37:28   
7        30          2  1297870702            河北 沧州  2019-06-27 22:34:16   
8        24          1  6128247211            福建 厦门  2019-06-27 17:50:34   
9        28          1  3712218552            广东 深圳  2019-06-27 17:39:09   
10       53          1  5532084803               香港  2019-05-23 09:57:11   
11       27          1  6310463353               四川  2019-06-27 23:14:00   
12       31 

   view_num inform_num reported_id reported_address            post_time  \
0       110         13  5934497728               海外  2019-06-16 09:27:13   
1       351          1  2156393955               北京  2019-06-16 12:03:39   
2        41          1  5874715391               安徽  2019-06-15 06:17:02   
3        69         13  1676764137               北京                 None   
4       919        311  1502981365               香港  2019-06-17 00:29:15   
5        22          1  3263783662               广东  2019-06-12 13:16:21   
6        93          2  1953042082               江苏  2019-06-15 14:43:54   
7        26          1  3315248087               北京  2019-06-12 14:55:17   
8        46         35  7081639316            甘肃 兰州  2019-06-18 01:47:16   
9        33          1  5224108583            香港 其他  2019-06-18 10:05:33   
10       74          1  7066517553               其他  2019-06-18 01:23:08   
11       40          5  1207540334            江苏 南京                 None   
12       52 

...

   view_num inform_num reported_id reported_address            post_time  \
0       107          1  6617113294               山东  2019-03-22 16:27:28   
1      1034          8  6124881270           江苏 连云港                 None   
2      1072          1  3748828515               其他                 None   
3       180          1  1355658213            浙江 宁波  2019-03-17 13:16:33   
4       905         35  2863918220            安徽 合肥                 None   
5       150          3  3631945211           北京 朝阳区  2019-03-24 23:49:18   
6       139          7  5310905955            辽宁 大连  2019-03-02 19:13:50   
7       123          1  2683179195               海外  2019-02-28 17:55:42   
8       296         29  3316281682               山东  2019-03-23 12:40:03   
9       216          1  3486117517               上海  2019-03-29 17:37:44   
10      129          1  3269886993               上海                 None   
11      115          1  5870138674               上海                 None   
12      115 

   view_num inform_num reported_id reported_address            post_time  \
0        46          1  1668646895           北京 东城区  2019-03-14 19:35:44   
1        41          3  1746105344            江苏 苏州  2019-03-15 19:42:31   
2        39          1  2456588133           海外 新加坡  2019-03-15 11:02:08   
3        40          2  6710143719               其他  2019-03-15 09:24:50   
4        43          1  5079906960           河北 秦皇岛  2019-03-14 17:22:21   
5        37          1  1294423027            浙江 衢州  2019-03-13 12:56:38   
6        52          1  1872728614               浙江  2019-03-13 22:43:35   
7        41          1  2516427265           北京 东城区  2019-03-15 18:26:30   
8        49          1  2830145714               江苏                 None   
9        74          7  5571681812               福建  2019-03-15 16:30:13   
10       44          1  1008762772           河北 秦皇岛  2019-03-15 18:46:44   
11       52          1  6270098657               浙江  2019-03-15 18:20:50   
12       47 

   view_num inform_num reported_id reported_address            post_time  \
0        46          1  1293758223           北京 东城区  2019-03-13 15:56:49   
1        44          1  5825099864               其他  2019-03-13 19:44:29   
2       227          3  5597100240           北京 朝阳区  2019-03-14 10:05:40   
3        48          2  7010208611               四川                 None   
4        39          1  2141533722            福建 厦门  2019-03-14 07:48:38   
5        40          4  5913955212               其他                 None   
6        39          1  3948180503            安徽 合肥  2019-03-14 10:01:33   
7        40          1  6874982571               山东  2019-03-14 10:28:02   
8        40          2  1403418835               天津  2019-03-14 13:51:19   
9        40          2  1736049753            河北 沧州  2019-03-14 10:10:19   
10       42          1  5414792460           海外 新加坡  2019-03-12 20:36:38   
11       39          1  1403418835               天津  2019-03-14 14:07:08   
12       47 

   view_num inform_num reported_id reported_address            post_time  \
0        57          2  6469995952               其他                 None   
1        73          1  5649968612               其他                 None   
2        51          3  1608707967               其他  2019-03-12 23:27:15   
3        37          1  5588225818               香港  2019-03-11 19:16:38   
4        37          1  1910791397               广东  2019-03-12 10:34:45   
5        39          1  2262417170               四川  2019-03-13 09:37:06   
6        39          3  5643754422               北京  2019-03-13 12:50:12   
7        38          1  1912190685               河南  2019-03-13 15:27:30   
8        64          4  2199841250               浙江  2019-03-13 16:02:59   
9        40          2  5643754422               北京  2019-03-13 12:51:49   
10       40          1  1235654591            湖南 益阳                 None   
11       54          2  1280289893              黑龙江                 None   
12       41 

   view_num inform_num reported_id reported_address            post_time  \
0        50          1  1539610161           上海 静安区                 None   
1        47          1  3429647542           北京 东城区  2019-02-11 09:01:30   
2        52          2  1715768694            湖南 长沙  2019-02-11 10:19:27   
3        46          1  1594390633               其他  2019-02-11 07:55:21   
4        70          1  5543374675            山东 临沂                 None   
5        50          3  5684047276            辽宁 铁岭  2019-02-10 20:56:37   
6        47          1  5134966097            四川 绵阳  2019-02-12 08:00:44   
7        64          1  6447843471               四川  2019-02-11 18:50:08   
8        48          4  2113191125            河北 廊坊  2019-02-10 22:58:50   
9        52          1  3286227294            四川 成都  2019-02-12 07:53:38   
10       55          1  5284120033               其他                 None   
11       88          1  3395406160               河南  2019-02-06 15:25:52   
12       51 

In [29]:
path = 'text_data/2.txt'
html = open(path, 'r', encoding='utf-8')
htmlhandle = html.read()
soup = BeautifulSoup(htmlhandle, 'lxml')

In [30]:
count = 0
result = pd.DataFrame({},index=[0])

In [31]:
result.drop(result.index,inplace=True)
result = pd.DataFrame()

In [34]:
view = []
inform = []
reported_id = []
reported_address = []
reported_gender = []
reported_credit = []
post_time = []
cn_links = []
## 访问次数
li = soup.find_all(text=re.compile("访问次数"))
pattern = re.compile(r'\d+')
for item in li:
    string = unidecode.unidecode(item.string)
    view.append(re.findall(pattern, string)[0])
## 举报人数
list_report = soup.select('input[node-type="uids_num"] ')
for item in list_report:
    inform.append(item['value'])

for item in soup.find_all("div", class_="user bg_orange2 clearfix"):
    ## 被举报人id
    link_u = item.select('p a')
    reported_id.append(str(link_u)[29:39])
    
    ## 被举报人位置
    string = str(item.contents[5])[-15:]
    add = re.sub("[A-Za-z0-9\<\>\"\/\\t]", "", string)
    reported_address.append(add)
    
    ## 被举报人信用等级
    x = item.select('p a[target="_blank"] ')
    credit_u_i = re.findall(r'信用等级：(.+)"' ,str(x))
    reported_credit.append(credit_u_i)
    
## 原微博发布时间
'''for item in soup.find_all(text=re.compile("发布时间")):
    match=re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', item)
    if match==None:
        post_time.append('nn')
    else:
        post_time.append(time.strptime(match.group(), '%Y-%m-%d %H:%M:%S'))'''
for item in soup.find_all('div', {'class': 'item top'}):
    if item.p.a == None:
        post_time.append('None')
    else:
        match = re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',str(item.p))
        post_time.append(match[0])
        #post_time.append(time.strptime(match.group(), '%Y-%m-%d %H:%M:%S'))

## 原微博链接
for item in soup.select('div[class="W_main_half_r"] div div div div[class="item top"]'):
    if item.p.a==None:
        cn_links.append('None')
    else:
        weibo_cn = re.findall(r'(?<=\/)[\w]*(?=\"\s)', str(item.p.a))
        cn_links.append('https://weibo.cn/comment/' + weibo_cn[0])

    
result['view_num'] = view
result['inform_num'] = inform
result['reported_id'] = reported_id
'''result['reported_gender'] = reported_gender'''
result['reported_address'] = reported_address
result['post_time'] = post_time
result['link_to_post'] = cn_links
result['reported_creidt'] = reported_credit

In [4]:
result

,view_num,inform_num,reported_id,reported_address,post_time,link_to_post,reported_credit
0,59,1,1598954802,辽宁,None,nn,[中]
1,58,1,1805611493,天津,None,nn,[中]
2,65,1,2517144137,河南 郑州,None,nn,[中]
3,66,1,5071460042,北京,None,nn,[中]
4,75,1,6607181111,山西,2019-01-08 14:06:49,https://weibo.cn/comment/HaZkqDXtx,[中]
5,71,1,3791688344,北京,None,nn,[中]
6,186,1,1853734181,北京,None,nn,[中]
7,80,1,5386362398,陕西,2019-01-09 19:18:20,https://weibo.cn/comment/HbaNnd9Rr,[正常]
8,101,1,1307191010,安徽,2019-01-10 13:10:16,https://weibo.cn/comment/HbhOtgk5i,[正常]
9,73,3,5570283618,江西,2019-01-15 09:41:33,https://weibo.cn/comment/Hc1Af6XRM,[中]


In [9]:
crawl_path = 'crawl_data/crawl' + str(1) + '.csv'
result.to_csv (crawl_path, index = None, header=True)

In [26]:
soup.find_all(text=re.compile("发布时间"))

['   \t被举报微博   \t   \t\t发布时间：2019-08-09 20:40:30   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-11 16:04:27   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-11 14:58:05   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-10 21:39:42   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-12 09:08:58   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-12 16:54:45   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-05-26 10:55:30   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-12 15:57:17   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-06-09 09:03:09   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-11 09:23:57   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-12 07:56:22   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-20 07:15:09   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-19 23:49:16   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-04 10:34:18   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-04 10:35:02   \t   \t',
 '   \t被举报微博   \t   \t\t发布时间：2019-08-04 15:17:43   \t   \t',
 '   \t被举报微博   \t   \t\t